In [23]:
import sqlite3

connexion = sqlite3.connect('base.db')
curseur = connexion.cursor()

# Création de la table Températures
curseur.execute(""" CREATE TABLE Temperatures(
                    date TEXT PRIMARY KEY UNIQUE, 
                    temperature_moyenne INTEGER
                )
                """)

# Création de la table Gares
curseur.execute(""" CREATE TABLE Gares(
                    code_uic INTEGER,
                    nom_gare TEXT,
                    latitude REAL,
                    longitude REAL,
                    frequentation_2019 INTEGER,
                    frequentation_2020 INTEGER,
                    frequentation_2021 INTEGER
                )
                """)

# Création de la table objets trouvés
curseur.execute(""" CREATE TABLE Objets_trouves (
                    id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE,
                    date TEXT,
                    type TEXT,
                    gare TEXT,
                    code UIC INTEGER,
                    FOREIGN KEY (gare) REFERENCES Gares(nom_gare),
                    FOREIGN KEY (date) REFERENCES Temperatures(date)
                    )""")


connexion.commit()
connexion.close()

OperationalError: unable to open database file

In [ ]:
connexion = connexion = sqlite3.connect('base.db')
curseur = connexion.cursor()

curseur.execute("""DROP INDEX IF EXISTS idx_nom_gare_unique""")

connexion.commit()
connexion.close()

#### Requeter pour avoir une base de données complète

In [ ]:
import requests
import pandas as pd
import sqlite3
import urllib.parse


url1 = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=gc_obo_gare_origine_r_name+%3D+%22"
url2 = "&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&timezone=Europe%2FParis&rows=-1"


gares = ['Paris Gare de Lyon', 'Paris Montparnasse', 'Paris Gare du Nord', 'Paris Saint-Lazare', 'Paris Est', 'Paris Bercy', 'Paris Austerlitz']
date =[2019, 2020,2021,2022]

data_frames = []
for g in gares:
    for d in date:
        api_url = f"{url1}{urllib.parse.quote(g)}%22+AND+date%3D{str(d)}{url2}"
        response = requests.get(api_url)
        data = response.json()
        records = data["records"]
        record_fields = [record["fields"] for record in records]
        df = pd.DataFrame(record_fields)
        data_frames.append(df)

df_concatenated = pd.concat(data_frames, ignore_index=True)


# Créer un DataFrame à partir des champs des enregistrements
df = pd.DataFrame(df_concatenated)


# Écrire le DataFrame dans un fichier CSV
df.to_csv("objets-trouves.csv", index=False)

# Créer une connexion à la base de données
connexion = sqlite3.connect('bdd.db')

df.to_sql('objets_trouves', connexion,if_exists='replace', index=False)

connexion.close()


In [ ]:
import requests
import sqlite3

url_gares = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=gare_alias_libelle_noncontraint='Paris'&sort=gare_alias_libelle_noncontraint&rows=-1"


response = requests.get(url_gares)

gares_coord_list = []

if response.status_code == 200:
    data = response.json()
    for record in data['records']:
        code_uic = record['fields']['uic_code']
        gare_alias_libelle_noncontraint = record['fields']['gare_alias_libelle_noncontraint']
        if 'wgs_84' in record['fields']:
            latitude = record['fields']['wgs_84'][0]
            longitude = record['fields']['wgs_84'][1]
            gares_coord_list.append([code_uic,gare_alias_libelle_noncontraint, latitude, longitude])

            
else:
    print("Une erreur s'est produite lors de la requête à l'API.")
    

connexion = sqlite3.connect("base.db")
curseur = connexion.cursor()
for gare in gares_coord_list:
    code_uic = gare[0]
    nom_gare = gare[1]
    latitude = gare[2]
    longitude = gare[3]
    curseur.execute("INSERT INTO Gares (code_uic,nom_gare, latitude, longitude) VALUES (?,?, ?, ?)", (code_uic, nom_gare, latitude, longitude))
connexion.commit()
connexion.close()

In [ ]:
import requests
import sqlite3
import pandas as pd 

url_frequentation_gares = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=frequentation-gares&q=nom_gare%3D%27Paris%27&sort=nom_gare&rows=-1"


response = requests.get(url_frequentation_gares)

gare_frequentation_list = []

if response.status_code == 200:
    data = response.json()
    for record in data['records']:
        gare = record['fields']['nom_gare']
        frequentation_2019 = record['fields']['total_voyageurs_non_voyageurs_2019']
        frequentation_2020 = record['fields']['total_voyageurs_non_voyageurs_2020']
        frequentation_2021 = record['fields']['total_voyageurs_non_voyageurs_2021']
        gare_frequentation_list.append([gare,frequentation_2019,frequentation_2020,frequentation_2021])
        

            
else:
    print("Une erreur s'est produite lors de la requête à l'API.")
    
connexion = sqlite3.connect("base.db")
curseur = connexion.cursor()
for gare in gare_frequentation_list:
    nom_gare = gare[0]
    frequentation_2019 = gare[1]
    frequentation_2020 = gare[2]
    frequentation_2021 = gare[3]
    
    curseur.execute("UPDATE Gares SET frequentation_2019= ?,frequentation_2020 = ?, frequentation_2021 = ? WHERE nom_gare=  ?", (frequentation_2019, frequentation_2020, frequentation_2021,nom_gare))
connexion.commit()
connexion.close()

connexion.close()

#---------------------------------------------------------------------------

connexion = sqlite3.connect("base.db")
curseur = connexion.cursor()
curseur.execute("""DELETE FROM Gares
WHERE rowid NOT IN (
  SELECT MIN(rowid)
  FROM Gares
  GROUP BY nom_gare, latitude, longitude
)
""")
connexion.commit()
connexion.close()

In [ ]:
import requests
import pandas as pd
import sqlite3
import datetime

# Clé d'API de worldweatheronline
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY_TEMP = os.getenv("API_KEY_TEMP")


# Coordonnées de Paris
COORDS = "48.8566,2.3522"

# Définition de la période de temps
start_date = datetime.date(2019, 1, 1)
end_date = datetime.date(2022, 12, 31)
delta = datetime.timedelta(days=35)

# Liste pour stocker les données de température
temperature_dates_list = []

# Boucle pour récupérer les données de température par période de 35 jours
while start_date <= end_date:
    end_period = start_date + delta
    if end_period > end_date:
        end_period = end_date
    url = f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?q={COORDS}&date={start_date}&enddate={end_period}&tp=24&format=json&key={API_KEY_TEMP}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for weather in data['data']['weather']:
            date = weather['date']
            temperature_moyenne = weather['avgtempC']
            temperature_dates_list.append([date,temperature_moyenne])
    else:
        print("Une erreur s'est produite lors de la requête à l'API.")
    start_date += delta

for entree in temperature_dates_list:
    gare = entree[0]
    temperature = entree[1]
    
    curseur.execute("UPDATE Gares SET frequentation_2019= ?,frequentation_2020 = ?, frequentation_2021 = ? WHERE nom_gare=  ?", (frequentation_2019, frequentation_2020, frequentation_2021,nom_gare))

# Fermeture de la connexion à la base de données
connexion.close()
